In [303]:
import json
f = open("cmd_setumei.json")
setumei = json.load(f)

In [316]:
n = 3
state = []

def create_state(cmd_row):
    cmd = []
    index = 0
    for item in cmd_row:
        if item[0] != "[":
            cmd.append({
                "val":item,
                "args":[]
            })
            index += 1
        else:
            cmd[index-1]["args"].append({
                "placeholder":item.replace("]","").replace("[","")
            })
        
    
    for i in range(len(cmd)):
        if not cmd[i]["val"] in setumei:
            print(cmd[i]["val"])
            description = input()
            setumei[cmd[i]['val']] = description
        s = []
        for j in range(i,min(i+n,len(cmd))):
            s.append(cmd[j])
        
        s = [v for v in s if len(v) >= 2]
        if len(s) >= 2:
            state.append(s)

with open("cmd_data.txt","r") as f:
    for row in f:
        cmd = row.split()
        create_state(cmd)

cd


 ディレクトリの移動


touch


 ファイルの作成


clear


 画面の表示を消去


pwd


 カレントディレクトリの表示


In [305]:
ng = {}
def create_ngram(states):
    
    for cmd in states:
        path = ng
        for i in range(len(cmd)):
            if cmd[i]['val'] in path:
                path[cmd[i]['val']]["count"] += 1
            else:
                if cmd[i]['val'] == '|':
                    cmd_type = "pipe"
                elif cmd[i]['val'][0] == "-":
                    cmd_type = "option"
                elif cmd[i]['val'] in ["<<","<",">>",">"]:
                    cmd_type = "redirect"
                else:
                    try:
                        result = subprocess.check_output(["type",cmd[i]['val']])
                        if i == 0:
                            cmd_type = "command"
                        else:
                            if cmd[i-1]['val'] ==  "|":
                                cmd_type = "command"
                            else:
                                
                                cmd_type = "option"
                            
                    except Exception as e:
                        cmd_type = "option"

                if cmd[i]['val'] in setumei:
                    description = setumei[cmd[i]['val']]
                else:
                    print("target : ",cmd[i])
                    description = input()
                    setumei[cmd[i]['val']] = description
                
                path[cmd[i]['val']] = {
                    "type":cmd_type,
                    "count":1,
                    "description":description,
                    "cmd":cmd[i],
                    "next":{}
                }

            path = path[cmd[i]['val']]["next"]
        



In [306]:
create_ngram(state)

target :  {'val': '-r', 'args': [{'placeholder': 'ディレクトリ名'}]}


 ディレクトリを削除


target :  {'val': '-f', 'args': [{'placeholder': 'ファイル名'}]}


 エラーメッセージを表示しない


In [307]:
import json
model = json.dumps(ng,ensure_ascii=False,indent=2)
with open("model.json","w") as f:
    f.write(model)

In [317]:
import json
des = json.dumps(setumei,ensure_ascii=False,indent=2)
with open("cmdDescription.json","w") as f:
    f.write(des)

In [309]:
import subprocess

In [311]:
ng["cat"]

{'type': 'command',
 'count': 3,
 'description': 'ファイルを出力',
 'cmd': {'val': 'cat', 'args': [{'placeholder': 'ファイル名'}]},
 'next': {'>>': {'type': 'redirect',
   'count': 1,
   'description': 'コマンドの結果をファイルに書き込む(追加)',
   'cmd': {'val': '>>', 'args': [{'placeholder': '追加対象'}]},
   'next': {}},
  '>': {'type': 'redirect',
   'count': 1,
   'description': 'ファイルの末尾に追加(上書き）',
   'cmd': {'val': '>', 'args': [{'placeholder': '上書き対象'}]},
   'next': {}},
  '|': {'type': 'pipe',
   'count': 1,
   'description': '前のコマンドの結果を次に渡す',
   'cmd': {'val': '|', 'args': []},
   'next': {'grep': {'type': 'command',
     'count': 1,
     'description': 'ファイルから文字列の検索',
     'cmd': {'val': 'grep', 'args': [{'placeholder': '検索ワード'}]},
     'next': {}}}}}}